##### Import Libraries

In [1]:
import requests # To make connection with API
import pandas as pd 
import geopandas as gpd # To read shapefile 
import json # To save API response data into json file
import logging # To create a log file to track progress or highlight issues

/usr/local/Caskroom/miniconda/base/envs/rs/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


#####  Create a Log File

In [3]:
# Name your log file
LOG_FILE = 'Script_2_soil_layers.log'    

# Create a log file (and set configuration to INFO)
  # Note: Wont overwrite if log file exists
logging.basicConfig(filename= LOG_FILE,level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

#####  Load API Credentials

In [4]:
CREDENTIALS_FILE = 'credentials.json'

# Load API Credentials
with open(CREDENTIALS_FILE, 'r') as file:
    CREDENTIALS = json.load(file) # dictionary dtype

### Section 1: Prepare API Request Data

##### Load Shapefile

In [5]:
# State your existing shapefile location (directory)
SHAPEFILE_DIR = "layers/POLYGON.shp"   

# Load shapefile
SHAPEFILE_DF = gpd.read_file(SHAPEFILE_DIR) # geo df
SHAPEFILE_DF.head()

,geometry
0,"POLYGON ((84.01875 26.68606, 84.01888 26.68566..."
1,"POLYGON ((78.58184 29.08218, 78.58193 29.08213..."
2,"POLYGON ((71.58564 25.78761, 71.58641 25.78688..."
3,"POLYGON ((78.93822 27.29710, 78.93849 27.29688..."
4,"POLYGON ((78.77363 24.02042, 78.77419 24.02104..."


##### F1 - Modify dataframe

In [6]:
# Name your csv file
CSV_FILENAME = 'Script_2_modified_farms_df.csv'  

In [7]:
def modify_df(shapefile_df):
    """ Add four new columns to the shapefile dataframe.
    
    The four new columns represents a farm's centroid, longitude, latitude and farm index.
    
    Params:
        (geosdataframe): The shapefile dataframe retrieved from reading shapefile.

    Returns:
        (geosdataframe): modified shapefile dataframe
        
    """
    # Create 3 new columns -'centroid','longitude','latitude' with data.
    shapefile_df['centroid'] = shapefile_df.centroid # Column with centroid data. 
    shapefile_df['longitude']= shapefile_df.centroid.x # Column with longtitude data
    shapefile_df['latitude'] = shapefile_df.centroid.y # Column with latitude data
    
    ## Create Data for 4th columns
    # Create an empty list to later store farm names. A list of string objects.
    farm_index = []
    
    for i in shapefile_df.index:
        farm_id = 'farm_{0}'.format(i+1)
        farm_index.append(farm_id)
    
    # Create 4th new column - 'farm_index' with data
    shapefile_df['farm_index'] = farm_index # Column with farm_index data
    
    # Assign new name to updated df
    modified_df = shapefile_df
    
    # Save modified dataframe as CSV file (named at beginning of script)
    modified_df.to_csv(CSV_FILENAME) 
    
    return modified_df # modified df with 4 new columns

In [8]:
# Call Functiion 1
MODIFIED_DF = modify_df(SHAPEFILE_DF)
MODIFIED_DF.head()

/var/folders/z8/lp4tjxbd48v4dbw1yqd037lm0000gn/T/ipykernel_11647/713976719.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shapefile_df['centroid'] = shapefile_df.centroid # Column with centroid data.
/var/folders/z8/lp4tjxbd48v4dbw1yqd037lm0000gn/T/ipykernel_11647/713976719.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shapefile_df['longitude']= shapefile_df.centroid.x # Column with longtitude data
/var/folders/z8/lp4tjxbd48v4dbw1yqd037lm0000gn/T/ipykernel_11647/713976719.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shapefile_df['latitude'] = shapefi

,geometry,centroid,longitude,latitude,farm_index
0,"POLYGON ((84.01875 26.68606, 84.01888 26.68566...",POINT (84.01912 26.68566),84.019123,26.685658,farm_1
1,"POLYGON ((78.58184 29.08218, 78.58193 29.08213...",POINT (78.58205 29.08281),78.582050,29.082805,farm_2
2,"POLYGON ((71.58564 25.78761, 71.58641 25.78688...",POINT (71.58665 25.78710),71.586647,25.787098,farm_3
3,"POLYGON ((78.93822 27.29710, 78.93849 27.29688...",POINT (78.93855 27.29719),78.938553,27.297185,farm_4
4,"POLYGON ((78.77363 24.02042, 78.77419 24.02104...",POINT (78.77369 24.02090),78.773687,24.020901,farm_5


### Section 2 - Create Functions to Make Connection with GEOS API

##### F2 - First API call to Get Soil Layers

In [9]:
def get_layers(longitude, latitude, farm_name):
    """ First API call to GEOS API to get the soil layers on a farm.
    
    Using post method to make the API request to 'ground/wps/info' endpoint to acquire 
    the various soil layers (namely - cec, cfvo, clay, nitrogen, ocd, sand, silt, soc, 
    ocs, bdod, phh2o) of a single farm.
    
    Params:
        (float): Longtitude value
        (float): Latitude value
        (str): Give a name to the farm e.g. "Farm Rainbow".
        
    Returns:
        (dict): Response body from 'ground/wps/info' endpoint.
        
    """
    # Assign a variable to ground/wps/info endpoint (URL)
    wps_info_endpoint = 'https://abcd.xyz.aabb.xxyy/ground/wps/info'
    
    # Assign a variable to API Key (Py Dictionary)
    api_key_header = CREDENTIALS
       
    # Original GEOS-API request body content in JSON format/ Python Dictionary
    wps_info_request_body = {
      "longitude": longitude,
      "latitude": latitude,
      "version": "v1",
      "source": "sg"
    }
    
    # Send an API request using Post Method to'ground/wps/info' endpoint
    # And receive a response which is saved as variable, r.
    response = requests.post(url = wps_info_endpoint, 
                             headers = api_key_header, 
                             json = wps_info_request_body)

    # Parse the response body text as JSON Object (or Python Dictionary)
    # And assign a variable it.
    wps_info_Dict = response.json()
    
    # If response status code is not 200, raise an exception
    if response.status_code != 200:
        raise Exception(' ground/wps/info endpoint. API Status Code:{0}, Fail. Soil layers on {1} (Lon {2} Lat {3}).'.format(response.status_code,
                                                                                                                              farm_name, longitude, latitude))
        logging.info(' ground/wps/info endpoint. API Status Code:{0}, Fail. Soil layers on {1} (Lon {2} Lat {3}).'.format(response.status_code,
                                                                                                                           farm_name, longitude, latitude))
    # Otherwise, proceed to acquire soil layers 
    else:
        logging.info(' ground/wps/info endpoint. API Status Code:{0}, Success. Soil layers on {1} (Lon {2} Lat {3}).'.format(response.status_code,
                                                                                                                              farm_name, longitude, latitude))
    # Show the response body text as JSON Object (Python Dictionary)
    return wps_info_Dict

In [11]:
# ## Test Function2 with a pair of lat-lon coordinates. And assign a variable.
# WPS_INFO_DICT = get_layers(11.57307130825532,52.8382292779053,"Farm Rainbow")

# # Show output (Py Dictionary)
# WPS_INFO_DICT

##### F3 - Remove three soil layers, longitude and lattitude

In [12]:
def remove_layers(wps_info_Dict):
    """ Remove three soil layers, longitude and latitude.
    
    Currently, due to glitches with 3 soil layers (namely -'ocs','bdod','phh2o'), these 
    soil layers would be removed prior to making an API call to 'ground/wps' endpoint. 
    Also, the 'longitude'and 'latitude' dictionary keys are also not required.
    Hence, alter the original dictionary (acquired from 'ground/wps/info' endpoint) to only 
    have the interested Key-Value pairs.
    
    Params:
        (dict): original dictionary acquired from 'ground/wps/info' endpoint.
        
    Returns:
        (dict): altered dictionary with interested Key-Value Pairs.
    """
    
    # Removes following Keys - 'ocs', 'bdod', 'phh2o', 'longitude','latitude'
    keys_to_remove = ['ocs','bdod','phh2o','longitude','latitude']
    for key in keys_to_remove:
        del wps_info_Dict[key]
    
    # Rename dictionary after removal of keys
    alt_Dict = wps_info_Dict   
    
    # Outputs a dictionary of dicionaries
    return alt_Dict 

In [14]:
# # Test Function 3
# ALT_DICT = remove_layers(WPS_INFO_DICT)
# ALT_DICT

In [16]:
# # View the dictionary keys
# ALT_DICT.keys()
# list(ALT_DICT.keys())

##### F4 - Format contents to suit API request body

In [17]:
def format_contents(alt_Dict):
    """ Formatting of content to suit API request body.
    
    Extract the wanted contents, which are the filtered or eight soil layers (dict keys) 
    and its 'depths' and 'values' data (dict values) from the altered dictionary, to create
    a new dictionary format which would be suitable for making an API call later.
    
    Then, collate each dictionary (which represents a soil layer) into a list.
    
    Params:
        (dict): Altered dictionary is a dictionary containing dictionaries.
        
    Returns:
        (list): A list of dictionaries which represents soil layers.
    """
    # Get dict keys and convert into an iterable list with a new assigned name
    layers_list = list(alt_Dict.keys())
    
    # Create a new empty list to later store elements (of dictionary data type)
    layers_dict_list = []
    
    # For each element (soil layer) in the layers_list, 
    # create a dictionary and add it to the list
    for name in layers_list:
        
        # Dictionary format contains 3 Key-Value Pairs.
        dictionary = {
            "name":name,
            "depths":alt_Dict[name]['depths'], 
            "values":alt_Dict[name]['values']
        }
        
        # Append dicionary to the list created earlier
        layers_dict_list.append(dictionary)
    
    # Output a list containing dictionaries
    return layers_dict_list

In [19]:
# # Test Function 4
# LAYERS_DICT_LIST = format_contents(ALT_DICT)
# LAYERS_DICT_LIST # List of dictionaries

##### F5 - Second API call to get soil layers info

In [20]:
def get_soil_info(longitude, latitude, layers_dict_list, farm_name):
    """ Second API call to GEOS API to get a farm's (lon-lat point) soil layers info.

    Using post method to make an API request to 'ground/wps' endpoint. 
    Each API request only seek for one soil layer info.
    Eight API requests would be made in this function to gather all the eight soil layers 
    info on a single farm (i.e. a lon-lat point).
    
    The eight soil layers, whose info would be retrieved, are 'cec', 'cfvo', clay, nitrogen, 
    ocd, sand, silt, and soc.
    
    Note: When making an API request to 'ground/wps' endpoint, API request body is preferred 
    to seek for only one soil layer's info over all/multiple soil layers' info at any given 
    time to avoid API error i.e. 429 Error 'Limit Exceeded'.
       
    Params:
        (float): Longtitude value
        (float): Latitude value
        (list): A list of dictionaries
        (str): Give a name to the farm e.g. "Farm Rainbow".
        
    Returns:
        (dict): response body from 'ground/wps' endpoint.   
    
    """
    # Assign a variable to ground/wps/info endpoint (URL)
    wps_endpoint = 'https://abcd.xyz.aabb.xxyy/ground/wps'
    
    # Assign a variable to API Credentials
    api_key_header = CREDENTIALS
    
    # Create an empty list to later store every soil layer's info retrieved from API request
    soil_info_list = []    
    
    # Loop through each soil layer to make an API request
    for each_layer in layers_dict_list:
        
        # Original GEOS-API request body content in JSON format/ Python Dictionary
        wps_request_body = {
            "longitude": longitude,
            "latitude": latitude,
            "version": "v1",
            "source": "sg",
            "layers": [each_layer],
            "resolution": 10,
            "processingMode": "SEQUENTIAL"
        }

        # Send an API request and save the reponse as a new variable
        response = requests.post(url=wps_endpoint, 
                                 headers=api_key_header, 
                                 json=wps_request_body)
    
        # If response status code is not 200, then raise an exception
        if response.status_code != 200:
            raise Exception(' ground/wps endpoint. API Status Code:{0}, Fail. Info on soil layer {1}.'.format(response.status_code,longitude,latitude))
            logging.info(' ground/wps endpoint. API Status Code:{0}, Fail. Info on soil layer {1}.'.format(response.status_code,longitude,latitude))
        
        # Otherwise, save reponse body
        else:
            logging.info(' ground/wps endpoint. API Status Code:{0}, Success. Info on soil layer {1}'.format(response.status_code,each_layer['name']))   
            
            # Parse the response body text as JSON Object (or Py Dictionary). Assign var name.
            wps_output = response.json() 
            
            # Store soil layer's info into the list created earlier
            soil_info_list.append(wps_output)

    logging.info(" Completed soil data acquisition on {0}.".format(farm_name))           
    
    return soil_info_list # A list containing dictionaries

In [22]:
# # Test Function 5 
# SOIL_INFO_LIST = get_soil_info(11.57307130825532,52.8382292779053,LAYERS_DICT_LIST,"Farm Rainbow")
# # print(len(SOIL_INFO_LIST)) # 8 => represents 8 soil layers
# SOIL_INFO_LIST 

### Section 3: Activate GEOS API Soil Data Acquisition & Storage

##### F6 - Make API requests to acquire soil data of all farms

In [23]:
# Name your json files
JSON_FILENAME = "Script_2_all_farms_soil_layers_info.json" 

In [24]:
def get_soil_info_AllFarms(modified_df):
    """ Get soil layers info for every farm in the modified dataframe and save into json file.
    
    As this function loops through the modified dataframe of farms to make API requests 
    and through activating the functions defined earlier, the succesful API responses will 
    return the soil layers info for a single farm.
    
    And every farm's soil layers info would be collated into a python dictionary with 
    farm ID as its key and soil layers info as its corresponding value. This dictionary
    would then be saved as a json file.
    
    Params:
        (geosdataframe): The modified dataframe gotten from loading shapefile and modify it.

    Returns:
        (dict): A dictionary of farm index as key and its entire soil layers info as 
                its value.
    """
    # Create an empty dictionary to later contain all farms' soil layers info
    all_farms_soil_layers_info = {}
    
#     for index, row in df.iterrows()
    
    # Run through every farm's centroid to get its soil layers info
    for lon,lat,farm_id in zip(modified_df['longitude'],
                               modified_df['latitude'],
                               modified_df['farm_index']):
        wps_info_Dict = get_layers(lon, lat, farm_id)  # Call F1
        alt_Dict = remove_layers(wps_info_Dict) # Call F2
        layers_dict_list = format_contents(alt_Dict) # Call F3
        soil_info_list = get_soil_info(lon, lat, layers_dict_list, farm_id) # Call F4 
        
        # Append every farm's 'soil_info_list' result into dictionary
        all_farms_soil_layers_info[farm_id] = soil_info_list
        
    logging.info(" OPERATIONS COMPLETED - Completed soil data acquisition of all farms. Saved as json file.")
    
    # Save all farms' soil layers info into a json file
    with open(JSON_FILENAME, "w") as file:
        json.dump(all_farms_soil_layers_info, file)
    
    print(" OPERATIONS COMPLETED. Next, view soil data with 2 options.")

    # Output a dictionary
    return all_farms_soil_layers_info

In [25]:
# # Create a shorter df (less API requests) to test function 6. Otherwise, skip to line 5.
# MODIFIED_DF2 = MODIFIED_DF.copy() # Make a copy of the orginal modified df
# MODIFIED_DF2 = MODIFIED_DF2.iloc[7:8] # manual random selection of rows

# Call Function 6
ALL_FARMS_SOIL_LAYERS_INFO = get_soil_info_AllFarms(MODIFIED_DF)

 OPERATIONS COMPLETED. Next, view soil data with 2 options.


##### Two options to view result in this python script

In [ ]:
# # Option 1 - View output in a concise format
# ALL_FARMS_SOIL_LAYERS_INFO 

In [ ]:
# Option 2 - View output in a more readable format with indentation in this python script
# using 'json.dumps'method (serializing json -convert py dict to json object)
json_object = json.dumps(ALL_FARMS_SOIL_LAYERS_INFO, indent = 4) # str object
print(json_object)

##### F7 - Code for Reading the (results from) json file

In [ ]:
# def load_jsonfile():
#     """ Load json file into this python script to view its content
    
#     Params:
#         (str): json file directory e.g. "all_farms_soil_layers_info.json".

#     Returns:
#         (dict): A dictionary of farm index as key and its entire soil layers' info as 
#                 its value.
                
#     """
#     # Open json file
#     file = open(JSON_FILENAME)

#     # Load the content (dictionary) from json file
#     file_content = json.load(file)

#     # Close file
#     file.close()

#     # View content (dictionary)
#     return file_content

In [ ]:
# # Test Function 7
# FILE_CONTENT = load_jsonfile()
# FILE_CONTENT